# PINN for Laminar Flow over NACA 6412 Airfoil

This notebook solves the Navier-Stokes equations for flow over a NACA 6412 airfoil using a Physics-Informed Neural Network.

**Geometry**: NACA 6412
**Re**: 1000
**Domain**: $[-1, 2] \times [-1, 1]$

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path
import time

np.random.seed(42)
tf.random.set_seed(42)

# Parameters
Re = 1000.0
epochs = 2000 # Increase for better results
learning_rate = 1e-3
N_boundary = 1000
N_collocation = 5000

## 1. Geometry Generation (NACA 6412)
We need to generate the coordinates of the airfoil surface.

In [ ]:
def naca4(number, n_points=100):
    m = int(number[0]) / 100.0
    p = int(number[1]) / 10.0
    t = int(number[2:]) / 100.0

    x = np.linspace(0, 1, n_points)
    yt = 5 * t * (0.2969 * np.sqrt(x) - 0.1260 * x - 0.3516 * x**2 + 0.2843 * x**3 - 0.1015 * x**4)

    yc = np.where(x < p,
                  m / p**2 * (2 * p * x - x**2),
                  m / (1 - p)**2 * ((1 - 2 * p) + 2 * p * x - x**2))

    dyc_dx = np.where(x < p,
                      2 * m / p**2 * (p - x),
                      2 * m / (1 - p)**2 * (p - x))

    theta = np.arctan(dyc_dx)

    xu = x - yt * np.sin(theta)
    yu = yc + yt * np.cos(theta)
    xl = x + yt * np.sin(theta)
    yl = yc - yt * np.cos(theta)

    X = np.concatenate((xu, xl[::-1]))
    Y = np.concatenate((yu, yl[::-1]))
    
    return X, Y

# Generate Airfoil Points
X_airfoil, Y_airfoil = naca4("6412", n_points=200)

plt.figure(figsize=(10, 4))
plt.plot(X_airfoil, Y_airfoil, 'k')
plt.title("NACA 6412 Airfoil")
plt.axis('equal')
plt.grid()
plt.show()

## 2. Data Preparation
- **Inlet**: x=-1, u=1, v=0
- **Outlet**: x=2, p=0 (optional, or just rely on physics)
- **Walls**: y=1, y=-1, u=1, v=0 (Slip/Free stream)
- **Airfoil**: No-slip, u=0, v=0

In [ ]:
def get_dataset(N_b, N_c):
    # --- BOUNDARY POINTS ---
    # Inlet (x=-1)
    xy_inlet = np.hstack([np.full((N_b // 4, 1), -1.0), np.random.uniform(-1, 1, (N_b // 4, 1))])
    u_inlet = np.ones((N_b // 4, 1))
    v_inlet = np.zeros((N_b // 4, 1))

    # Outlet (x=2) - often we don't enforce strict u/v here, but let's enforce p=0 roughly or just continuity
    # For simplicity in this demo, we assume developed flow or free stream continue if far enough
    # Let's enforce Top/Bottom (y=1, y=-1) as free stream
    xy_top = np.hstack([np.random.uniform(-1, 2, (N_b // 4, 1)), np.full((N_b // 4, 1), 1.0)])
    u_top = np.ones((N_b // 4, 1))
    v_top = np.zeros((N_b // 4, 1))
    
    xy_bot = np.hstack([np.random.uniform(-1, 2, (N_b // 4, 1)), np.full((N_b // 4, 1), -1.0)])
    u_bot = np.ones((N_b // 4, 1))
    v_bot = np.zeros((N_b // 4, 1))

    # Airfoil Surface (No-slip)
    # Resample airfoil points to N_b // 4
    idx = np.random.choice(len(X_airfoil), N_b // 4)
    xy_foil = np.hstack([X_airfoil[idx].reshape(-1, 1), Y_airfoil[idx].reshape(-1, 1)])
    u_foil = np.zeros((N_b // 4, 1))
    v_foil = np.zeros((N_b // 4, 1))

    # Combine Boundary Data
    X_b = np.concatenate([xy_inlet[:,0:1], xy_top[:,0:1], xy_bot[:,0:1], xy_foil[:,0:1]])
    Y_b = np.concatenate([xy_inlet[:,1:2], xy_top[:,1:2], xy_bot[:,1:2], xy_foil[:,1:2]])
    U_b = np.concatenate([u_inlet, u_top, u_bot, u_foil])
    V_b = np.concatenate([v_inlet, v_top, v_bot, v_foil])

    # --- COLLOCATION POINTS ---
    # Uniform sampling in [-1, 2] x [-1, 1]
    X_c = np.random.uniform(-1, 2, (N_c, 1))
    Y_c = np.random.uniform(-1, 1, (N_c, 1))
    
    # Filter points inside airfoil (Approximation: use Matplotlib path or simple bounds)
    # For simplicity, we just keep all, or we could do a rough check. 
    # A robust way requires specific efficient checks, but let's assume the NN learns 0 inside naturally or we ignore physics there.
    # A simple polygon check:
    from matplotlib.path import Path
    path = Path(np.hstack([X_airfoil.reshape(-1, 1), Y_airfoil.reshape(-1, 1)]))
    mask = ~path.contains_points(np.hstack([X_c, Y_c]))
    X_c = X_c[mask]
    Y_c = Y_c[mask]

    return tf.cast(X_b, tf.float32), tf.cast(Y_b, tf.float32), tf.cast(U_b, tf.float32), tf.cast(V_b, tf.float32), tf.cast(X_c.reshape(-1,1), tf.float32), tf.cast(Y_c.reshape(-1,1), tf.float32)

X_b, Y_b, U_b, V_b, X_c, Y_c = get_dataset(N_boundary, N_collocation)

plt.figure(figsize=(10, 4))
plt.scatter(X_c, Y_c, s=1, c='gray', alpha=0.5)
plt.scatter(X_b, Y_b, s=5, c='red')
plt.axis('equal')
plt.show()

In [ ]:
class PINN(tf.keras.Model):
    def __init__(self):
        super(PINN, self).__init__()
        self.hidden_layers = [
            tf.keras.layers.Dense(64, activation='tanh'),
            tf.keras.layers.Dense(64, activation='tanh'),
            tf.keras.layers.Dense(64, activation='tanh'),
            tf.keras.layers.Dense(64, activation='tanh'),
            tf.keras.layers.Dense(64, activation='tanh')
        ]
        self.out = tf.keras.layers.Dense(3) # u, v, p

    def call(self, inputs):
        x = inputs
        for layer in self.hidden_layers:
            x = layer(x)
        return self.out(x)

@tf.function
def physics_loss(model, x, y):
    with tf.GradientTape(persistent=True) as tape2:
        tape2.watch(x)
        tape2.watch(y)
        with tf.GradientTape(persistent=True) as tape1:
            tape1.watch(x)
            tape1.watch(y)
            
            inputs = tf.concat([x, y], axis=1)
            outputs = model(inputs)
            u = outputs[:, 0:1]
            v = outputs[:, 1:2]
            p = outputs[:, 2:3]
        
        u_x = tape1.gradient(u, x)
        u_y = tape1.gradient(u, y)
        v_x = tape1.gradient(v, x)
        v_y = tape1.gradient(v, y)
        p_x = tape1.gradient(p, x)
        p_y = tape1.gradient(p, y)

    u_xx = tape2.gradient(u_x, x)
    u_yy = tape2.gradient(u_y, y)
    v_xx = tape2.gradient(v_x, x)
    v_yy = tape2.gradient(v_y, y)
    
    del tape1
    del tape2

    f_cont = u_x + v_y
    f_u = (u * u_x + v * u_y) + p_x - (1/Re) * (u_xx + u_yy)
    f_v = (u * v_x + v * v_y) + p_y - (1/Re) * (v_xx + v_yy)
    
    return tf.reduce_mean(tf.square(f_cont)) + \
           tf.reduce_mean(tf.square(f_u)) + \
           tf.reduce_mean(tf.square(f_v))

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model = PINN()

@tf.function
def train_step(x_b, y_b, u_b, v_b, x_c, y_c):
    with tf.GradientTape() as tape:
        # Boundary Loss (Dirichlet for U, V)
        out_b = model(tf.concat([x_b, y_b], axis=1))
        loss_b = tf.reduce_mean(tf.square(out_b[:, 0:1] - u_b)) + tf.reduce_mean(tf.square(out_b[:, 1:2] - v_b))
        
        # Physics Loss
        loss_p = physics_loss(model, x_c, y_c)
        
        loss = loss_b + loss_p
        
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss, loss_b, loss_p

In [ ]:
print("Starting Training...")
history = []
for epoch in range(epochs):
    l, lb, lp = train_step(X_b, Y_b, U_b, V_b, X_c, Y_c)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Loss {l.numpy():.4f} (BC: {lb.numpy():.4f}, Phy: {lp.numpy():.4f})")
    history.append(l.numpy())

## 3. Visualization

In [ ]:
n_vis = 200
x_vis = np.linspace(-1, 2, n_vis)
y_vis = np.linspace(-1, 1, n_vis)
X_grid, Y_grid = np.meshgrid(x_vis, y_vis)
X_flat = X_grid.flatten()[:, None]
Y_flat = Y_grid.flatten()[:, None]

preds = model(tf.cast(np.hstack([X_flat, Y_flat]), tf.float32))
u_vis = preds[:, 0].numpy().reshape(n_vis, n_vis)
v_vis = preds[:, 1].numpy().reshape(n_vis, n_vis)
p_vis = preds[:, 2].numpy().reshape(n_vis, n_vis)
speed = np.sqrt(u_vis**2 + v_vis**2)

# Mask out airfoil for prettier plots
path = Path(np.hstack([X_airfoil.reshape(-1, 1), Y_airfoil.reshape(-1, 1)]))
mask_vis = path.contains_points(np.hstack([X_flat, Y_flat])).reshape(n_vis, n_vis)
speed = np.where(mask_vis, np.nan, speed)
u_vis = np.where(mask_vis, np.nan, u_vis)
v_vis = np.where(mask_vis, np.nan, v_vis)
p_vis = np.where(mask_vis, np.nan, p_vis)

plt.figure(figsize=(18, 6))

# Velocity Plot
plt.subplot(1, 2, 1)
plt.contourf(X_grid, Y_grid, speed, 100, cmap='jet')
plt.colorbar(label='Velocity Magnitude')
plt.streamplot(X_grid, Y_grid, u_vis, v_vis, density=1.5, color='white', linewidth=0.5)
plt.fill(X_airfoil, Y_airfoil, 'k')
plt.title("Velocity (Re=1000)")
plt.axis('equal')

# Pressure Plot
plt.subplot(1, 2, 2)
plt.contourf(X_grid, Y_grid, p_vis, 100, cmap='RdBu_r')
plt.colorbar(label='Pressure')
plt.fill(X_airfoil, Y_airfoil, 'k')
plt.title("Pressure Field")
plt.axis('equal')

plt.tight_layout()
plt.show()